In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from platform import python_version
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as T
import torch.nn.functional as F
import torch.utils.data as data_utils
import matplotlib.animation as animation
from IPython.display import HTML

In [4]:
!pip install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for wget: started
  Running setup.py install for wget: finished with status 'done'


  DEPRECATION: wget is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Preperation
Downloading the dataset from http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/zipped%20images/part2.zip, this might take several minutes since the dataset is about 5 GB.

In [11]:
import wget
url = "http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/zipped images/part2.zip"
filename = wget.download(url)
print(filename)

http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/zipped images/part2.zip
part2.zip


In [48]:
import zipfile
origin_file_path = "origin"

with zipfile.ZipFile(filename,"r") as zip_ref:
    zip_ref.extractall(origin_file_path)

In [59]:
# import cv2
import PIL
from tqdm import tqdm
masked_file_path = "masked"
if not os.path.exists(masked_file_path):
    os.mkdir(masked_file_path)

for image_file in tqdm(os.listdir(origin_file_path)):
    input_path = os.path.join(origin_file_path, image_file)
    if not os.path.isfile(input_path):
        print(f"{input_path} is not a file, skipping...")
        continue
    img = PIL.Image.open(input_path)
    img_ary = np.array(img, dtype=np.uint8)
    center = (640, 512)
    width, height = 400, 300
    startx, starty = center[0] - width//2, center[1] - height//2
    img_ary[starty:starty+height, startx:startx+width, :] = 0
    PIL.Image.fromarray(img_ary).save(os.path.join(masked_file_path, image_file))

100%|██████████| 6600/6600 [01:50<00:00, 59.71it/s]


## Some steps we can use for data preprocessing

In [ ]:
data_dir = "C:\\Users\\Настя\\GAN_images"
device = 'cuda'
batch_size = 128

In [ ]:
number_cats = len(os.listdir(data_dir + '\cats'))
print("Number of training data samples:", number_cats)

In [ ]:
# Chaining image transformations together using torchvision.transforms.Compose method
transform = T.Compose([
    T.Resize(img_size),
    T.CenterCrop(img_size),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Using next(iter(_)) we obtain a tensor with dimension (batch_size, num_channels, size, size)

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

images, labels = next(iter(dataloader))
images.shape

In [ ]:
fig, axis = plt.subplots(3, 5, figsize=(15,10))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        npimg = images[i].numpy()
        npimg = np.transpose(npimg, (1,2,0))
        ax.imshow(npimg)

## Here we will do data preprocessing with our data